<a href="https://colab.research.google.com/github/scarfboy/wetsuite-dev/blob/main/examples/collocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this notebook



Understanding what you can get out of rechtspraak.nl.

Note that this is mostly just showing our work. 

If the dataset we provides suits your needs you can use that, and this notebook would mostly just be a slower and more cumbersome way of making the same.

## Website, Open data API, and some other stuff

You are probably familiar with the [rechtspraak.nl](https://rechtspraak.nl) website and its [search that has a number of filters](https://uitspraken.rechtspraak.nl/#!/) (and some [exta query logic for the text](https://www.rechtspraak.nl/Uitspraken/Paginas/Hulp-bij-zoeken.aspx#1ab85aa0-e737-4b56-8ad5-d7cb7954718d77a998be-3c73-40e3-90f7-541fceeb00fd3)), which gives webpage results, with text where present.




There is also an open data API that exposes much the same in data form.
As [its documentation](https://www.rechtspraak.nl/SiteCollectionDocuments/Technische-documentatie-Open-Data-van-de-Rechtspraak.pdf)  mentions,
- you stick query parameters on the base URL of http://data.rechtspraak.nl/uitspraken/zoeken 
- the results mainly mention ECLIs, which you can fetch details for via e.g. https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:PHR:2011:BP5608


Worthy of note:
- the fields you can search mostly matches with the 'Uitgebreid zoeken' at uitspraken.rechtspraak.nl, like
  - instantie / court code (basically that third element in the ECLI)
  - rechtsgebied
  - procedure 

- **You can't search in the body text**.  This largely limits the API to a 'keep updated with new cases' feed.  
  - The interface behind the website search (queried via `https://uitspraken.rechtspraak.nl/api/zoek`) does, and sees like a better data API, it doesn't look like it's supposed to be used externally.

- there are plenty of cases where there is no text / document.  You can filter for this in the search.

- the identifiers used are ECLI (European Case Law Identifier)
  - which in this case will be mainly Dutch ECLIs  (`ECLI:NL:`...). 
    - which has absorbed the LJN (Landelijk Jurisprudentie Nummer) identifiers used before 2013.
  - court code XX (`ECLI:NL:XX:`...) is used for things not (yet?) assgined to a court, and/or non-Dutch ECLIs,
    - rechtspraak.nl may later resolve such ECLIs to a different ECLI. That makes their site the most up-to-date information, that a mirror may not necessarily be aware of (yet). 
    - Example: ECLI:NL:XX:2009:BJ4574 
      - in [XML metadata](https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:XX:2009:BJ4574) mentions it's now (isReplacedBy) ECLI:CE:ECHR:2009:0528JUD002671305 (note: )
      - in [webpage form](https://uitspraken.rechtspraak.nl/#!/details?id=ECLI:NL:XX:2009:BJ4574) also seems to link to [a place to find it](https://hudoc.echr.coe.int/eng#%7B%22ecli%22:%5B%22ECLI:CE:ECHR:2009:0528JUD002671305%22%5D%7D) (e.g. hudoc.echr.coe.int or e-justice.europa.eu).

In [3]:
import pprint, collections

import tqdm

import wetsuite.helpers.etree
import wetsuite.datacollect.rechtspraaknl
import wetsuite.helpers.koop_parse
import wetsuite.helpers.meta
import wetsuite.helpers.net
import wetsuite.helpers.localdata

### Example of search results

Amounts to title, summary, and pointing at the XML data

In [4]:
params = [ ('max', '20000'), ('return', 'DOC'),                       # DOC asks for things with body text only
           ('modified', '2023-03-01'), ('modified', '2023-04-01') ]   # date range    (the max easily means we only get a small amont of this, though)
search_results = wetsuite.datacollect.rechtspraaknl.search( params )
# search_results is a parsed etree object, so to see it relatively raw:
#print( wetsuite.helpers.etree.debug_pretty(search_results) ) 

# but our parsed form (each entry as a dict) is little different so we might as well show that:
entries = wetsuite.datacollect.rechtspraaknl.parse_search_results( search_results )
print( "Entries in results: %d\n"%len(entries) )
for entry in entries[:5]:
    pprint.pprint(entry)

https://data.rechtspraak.nl/uitspraken/zoeken?max=20000&return=DOC&modified=2023-03-01&modified=2023-04-01
Entries in results: 6628

{'ecli': 'ECLI:NL:RBZWB:2020:5807',
 'link': 'https://uitspraken.rechtspraak.nl/#!/details?id=ECLI:NL:RBZWB:2020:5807',
 'summary': '“Verdachte heeft een snijdende beweging tegen de buikstreek/de '
            'zij en een stekende beweging in de richting van de nek van het '
            'slachtoffer gemaakt met een schaar. Dit heeft geleid tot een '
            'lange schaafwond op de romp en een kleine schaafwond op het '
            'achterhoofd. Verdachte had de relatief botte papierschaar in het '
            'midden tussen de tweegedeeltes vast, waarbij de schaar openstond. '
            'Dit betekent dat he...',
 'title': 'ECLI:NL:RBZWB:2020:5807, Rechtbank Zeeland-West-Brabant, '
          '24-11-2020, 02-665079-19',
 'updated': '2023-02-28T23:01:29Z',
 'xml': 'https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:RBZWB:2020:5807'}
{'ecli': 'EC

## What does the data XML look like, and what can I easily do with it?

In [5]:
bytestring = wetsuite.helpers.net.download('https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:RBZWB:2020:5807')
tree  = wetsuite.helpers.etree.fromstring( bytestring )
print( wetsuite.helpers.etree.debug_pretty(tree) ) # print indented

<open-rechtspraak>
  <RDF>
    <Description>
      <identifier>ECLI:NL:RBZWB:2020:5807</identifier>
      <format>text/xml</format>
      <accessRights>public</accessRights>
      <modified>2023-03-01T00:01:29</modified>
      <issued label="Publicatiedatum">2020-11-24</issued>
      <publisher resourceIdentifier="http://rechtspraak.nl/">Raad voor de Rechtspraak</publisher>
      <language>nl</language>
      <creator resourceIdentifier="http://standaarden.overheid.nl/owms/terms/Rechtbank_Zeeland-West-Brabant" scheme="overheid.RechterlijkeMacht" label="Instantie">Rechtbank Zeeland-West-Brabant</creator>
      <date label="Uitspraakdatum">2020-11-24</date>
      <zaaknummer label="Zaaknr">02-665079-19</zaaknummer>
      <type resourceIdentifier="http://psi.rechtspraak.nl/uitspraak" language="nl">Uitspraak</type>
      <procedure resourceIdentifier="http://psi.rechtspraak.nl/procedure#peek" language="nl" label="Procedure">Peek</procedure>
      <procedure resourceIdentifier="http://psi.r

In [6]:
pprint.pprint( wetsuite.datacollect.rechtspraaknl.parse_content( tree ) )

{'creator': 'Rechtbank Zeeland-West-Brabant',
 'date': '2020-11-24',
 'identifier': 'ECLI:NL:RBZWB:2020:5807',
 'inhoudsindicatie': [([],
                       ['“Verdachte heeft een snijdende beweging tegen de '
                        'buikstreek/de zij en een stekende beweging in de '
                        'richting van de nek van het slachtoffer gemaakt met '
                        'een schaar. Dit heeft geleid tot een lange schaafwond '
                        'op de romp en een kleine schaafwond op het '
                        'achterhoofd. Verdachte had de relatief botte '
                        'papierschaar in het midden tussen de tweegedeeltes '
                        'vast, waarbij de schaar openstond. Dit betekent dat '
                        'het deel van de schaar dat het slachtoffer heeft '
                        'geraakt zodanig kort is geweest, dat het slachtoffer '
                        'niet zonder meer zwaar kon worden verwond. De kans '
                 

/usr/local/lib/python3.8/dist-packages/wetsuite/helpers/koop_parse.py:908: UserWarning: The behaviour of alineas_with_selective_path() is not fully decided, and may still change
  warnings.warn('The behaviour of alineas_with_selective_path() is not fully decided, and may still change')


## Fetching actual text

And, like in [datacollect_koop_docstructure_cvdr](datacollect_koop_docstructure_cvdr.ipynb) and [_bwb](datacollect_koop_docstructure_bwb.ipynb), 
let's point out there are [schemas](https://www.rechtspraak.nl/SiteCollectionDocuments/Schema-Open-Data-voor-de-Rechtspraak.zip) to the text's structure, and look at how they're followed.

In [7]:
rechtspraak_fetched = wetsuite.helpers.localdata.open_store('rechtspraak_fetched.db', key_type=str, value_type=bytes)

In [8]:
len( rechtspraak_fetched.keys() )

#rechtspraak_fetched.get( list(rechtspraak_fetched.keys())[0] )

3159723

In [9]:
paths = collections.defaultdict(int)

uitspraken, conclusies = 0,0


for entry in entries:
    entry_xml_url = 'https://data.rechtspraak.nl/uitspraken/content?id=%s'%entry['ecli']
    wetsuite.helpers.localdata.cached_fetch( rechtspraak_fetched, entry_xml_url)
    #bytestring = rechtspraak_fetched.get( entry_xml_url )
    
    tree  = wetsuite.helpers.etree.fromstring( bytestring )
    tree  = wetsuite.helpers.etree.strip_namespace( tree )
    #print( wetsuite.helpers.etree.debug_pretty(tree) )
    #pprint.pprint( wetsuite.datacollect.rechtspraaknl.parse_content( tree ) )

    conclusie = tree.find('conclusie')
    if conclusie is not None:
        for path, count in wetsuite.helpers.etree.path_count( conclusie ).items():
            paths[path] += count
        conclusies += 1

    uitspraak = tree.find('uitspraak')
    if uitspraak is not None:
        for path, count in wetsuite.helpers.etree.path_count( uitspraak ).items():
            paths[path] += count
        uitspraken += 1


In [10]:
print("Summarizing %d conclusies and %d uitspraken"%(conclusies, uitspraken))
for path, count in paths.items():
    #if count>200:
        print('%6d   %s'%(count, path))

Summarizing 0 conclusies and 6628 uitspraken
  6628   uitspraak/uitspraak.info/bridgehead
 46396   uitspraak/uitspraak.info/para
 59652   uitspraak/uitspraak.info/parablock/para
 39768   uitspraak/uitspraak.info/parablock
 13256   uitspraak/uitspraak.info/parablock/para/emphasis
  6628   uitspraak/uitspraak.info
 72908   uitspraak/section/title/nr
 72908   uitspraak/section/title
397680   uitspraak/section/para
 72908   uitspraak/section
258492   uitspraak/section/parablock/para
165700   uitspraak/section/parablock
 86164   uitspraak/section/parablock/para/emphasis
 46396   uitspraak/section/paragroup/nr
 72908   uitspraak/section/paragroup/parablock/para/emphasis
172328   uitspraak/section/paragroup/parablock/para
 92792   uitspraak/section/paragroup/parablock
 72908   uitspraak/section/paragroup/para
 46396   uitspraak/section/paragroup
  6628   uitspraak/section/paragroup/para/emphasis
 13256   uitspraak/section/paragroup/paragroup/nr
 26512   uitspraak/section/paragroup/paragroup/p

In [11]:
import wetsuite.helpers.koop_parse

# https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:RBROT:2022:4576

for entry in entries:
    entry_xml_url = 'https://data.rechtspraak.nl/uitspraken/content?id=%s'%entry['ecli']
    wetsuite.helpers.localdata.cached_fetch( rechtspraak_fetched, entry_xml_url)

    if b'uitspraak.info' in bytestring:
        print(entry['xml'])
    elif b'paragroup' in bytestring:
        print(entry['xml'])
    elif b'<section' in bytestring:
        print(entry['xml'])
    else:
        continue

    tree  = wetsuite.helpers.etree.fromstring( bytestring )
    tree  = wetsuite.helpers.etree.strip_namespace( tree )

    pprint.pprint( wetsuite.datacollect.rechtspraaknl.parse_content(tree) )
    #alinea_dicts = wetsuite.helpers.koop_parse.alineas_with_selective_path( tree, alinea_elemname='para' )
    #pprint.pprint( alinea_dicts ) 
    break


https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:RBZWB:2020:5807
{'creator': 'Rechtbank Zeeland-West-Brabant',
 'date': '2020-11-24',
 'identifier': 'ECLI:NL:RBZWB:2020:5807',
 'inhoudsindicatie': [([],
                       ['“Verdachte heeft een snijdende beweging tegen de '
                        'buikstreek/de zij en een stekende beweging in de '
                        'richting van de nek van het slachtoffer gemaakt met '
                        'een schaar. Dit heeft geleid tot een lange schaafwond '
                        'op de romp en een kleine schaafwond op het '
                        'achterhoofd. Verdachte had de relatief botte '
                        'papierschaar in het midden tussen de tweegedeeltes '
                        'vast, waarbij de schaar openstond. Dit betekent dat '
                        'het deel van de schaar dat het slachtoffer heeft '
                        'geraakt zodanig kort is geweest, dat het slachtoffer '
                     

It seems there are specific ideas about what should be in what structure (e.g. para in parablock in paragroup,  overview stuff in uitspraak.info)

In [20]:
import wetsuite.helpers.notebook
# use a helper that virualizes a selection within an etree, to see if our selection makes sense
# = 
wetsuite.helpers.notebook.etree_visualize_selection( example_tree, 'body/regeling/regeling-tekst//al', reindent=True, mark_subtree=True )

NameError: name 'example_tree' is not defined

### Count how many ECLIs have text in the first place

In [18]:
# from some quick counts, it seems like roughly 25% of URLs like https://data.rechtspraak.nl/uitspraken/content?id=<ecli>   have uitspraak text in their XML
#   though it seems there is variation over time

rechtspraak_urls = rechtspraak_fetched.keys()

print('COUNT from %d items'%len(rechtspraak_urls))
with_text, without_text    = 0, 0
for url in tqdm.tqdm( rechtspraak_urls):
    ecli = wetsuite.helpers.meta.findall_ecli( url )[0]   # assume there is just one  (it'd raise with an IndexError if there wasn't one)
    bytestring = rechtspraak_fetched.get( url )
    if b'<uitspraak ' in bytestring: # hackish optimization - tests whether there's uitpraak text in the document without parsing the XML
        with_text += 1
    else:
        without_text += 1

print( 'With text:%d (%d%%)   without:%d'%( with_text, (100.*with_text)/(with_text+without_text), without_text ) )

COUNT from 3159723 items


100%|██████████| 3159723/3159723 [01:15<00:00, 41867.80it/s]

With text:665910 (21%)   without:2493813


## Values for filters

In [ ]:
rgb = wetsuite.datacollect.rechtspraaknl.parse_rechtsgebieden()

# put it in a form that is a little easier to show as a tree
rechtsgebieden_groups = {}
for identifier, l in rgb.items():
    if len(l)==1:
        broad, = l
        rechtsgebieden_groups[broad] = {'identifier':identifier, 'name':broad, 'specific':[]}

    elif len(l)==2:
        specific, broad = l
        rechtsgebieden_groups[broad]['specific'].append( {'identifier':identifier, 'name':specific} )

rechtsgebieden_groups

In [32]:
# just a list, not sure how useful this is
ps = wetsuite.datacollect.rechtspraaknl.parse_proceduresoorten()

# put it in a form that is a little easier to show as a tree
proceduresoorten = {}
for d in ps:
    proceduresoorten[ d['Identifier'] ] = d['Naam'] 
proceduresoorten

{'http://psi.rechtspraak.nl/procedure#artikel81ROzaken': 'Artikel 81 RO-zaken',
 'http://psi.rechtspraak.nl/procedure#bodemzaak': 'Bodemzaak',
 'http://psi.rechtspraak.nl/procedure#cassatie': 'Cassatie',
 'http://psi.rechtspraak.nl/procedure#cassatieInHetBelangDerWet': 'Cassatie in het belang der wet',
 'http://psi.rechtspraak.nl/procedure#conservatoireMaatregel': 'Conservatoire maatregel',
 'http://psi.rechtspraak.nl/procedure#eersteAanlegEnkelvoudig': 'Eerste aanleg - enkelvoudig',
 'http://psi.rechtspraak.nl/procedure#eersteAanlegMeervoudig': 'Eerste aanleg - meervoudig',
 'http://psi.rechtspraak.nl/procedure#eersteEnEnigeAanleg': 'Eerste en enige aanleg',
 'http://psi.rechtspraak.nl/procedure#herziening': 'Herziening',
 'http://psi.rechtspraak.nl/procedure#hogerBeroep': 'Hoger beroep',
 'http://psi.rechtspraak.nl/procedure#hogerBeroepKortGeding': 'Hoger beroep kort geding',
 'http://psi.rechtspraak.nl/procedure#kortGeding': 'Kort geding',
 'http://psi.rechtspraak.nl/procedure#monde

See also [extras_datacollect_rechtspraak_codes](extras_datacollect_rechtspraak_codes.ipynb)

## Miscellanous notes

- URLs for an ECLI include
  - XML at https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:PHR:2011:BP5608
    - most interesting when you want case details as data
  - (there is a different XML form at https://uitspraken.rechtspraak.nl/api/document/?id=ECLI:NL:PHR:2011:BP5608 but this seems less useful than the previous)
  - the search results link to the case on the website, e.g. https://uitspraken.rechtspraak.nl/InzienDocument?id=ECLI:NL:PHR:2011:BP5608
    - which redirects to https://uitspraken.rechtspraak.nl/#!/details?id=ECLI:NL:PHR:2011:BP5608
    - this is useful when you want to link people to something useful based on an ECLI, but less interesting than the data
  - it seems the slightly shorter https://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:PHR:2011:BP5608 is equivalent
  - The website also often links to LiDo, e.g.  https://linkeddata.overheid.nl/document/ECLI:NL:PHR:2011:BP5608
    - If you want that as data, consider http://linkeddata.overheid.nl/service/get-links?ext-id=ECLI:NL:PHR:2011:BP5608&output=xml
    - though as https://linkeddata.overheid.nl/front/portal/services notes, this is not part of public LiDo so you'll need to request an account first

- Because there are over three million dutch ECLIs on record, getting a lot of data from here would take a while, and they ask you to be nice to their server.
  - There used to be a ZIP file (linked from [https://www.rechtspraak.nl/Uitspraken/paginas/open-data.aspx](https://www.rechtspraak.nl/Uitspraken/paginas/open-data.aspx) you could download to bootstrap your own copy, but [this data request](https://data.overheid.nl/community/datarequest/zip-bestand-alle-uitspraken) seems to confirm this was removed. As of this writing the URL to the ZIP file still works but they probably removed the link for a reason.
